In [2]:
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, BatchNormalization, Dropout, Flatten, Embedding, Flatten, Dense, LSTM
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D
from tensorflow.keras.preprocessing import sequence
import tensorflow.keras.utils
import numpy as np
import pandas as pd
import spacy
import spacy_transformers
import en_trf_bertbaseuncased_lg
import os
import tqdm

In [3]:
df = pd.read_csv('train.csv')

In [4]:
df.dropna(inplace=True)

In [5]:
df_test = pd.read_csv('test.csv')

In [6]:
X = df['text']
y = df['sentiment']

In [7]:
y.replace({'neutral':0, 'negative':-1, 'positive':1}, inplace=True)

In [8]:
y = tensorflow.keras.utils.to_categorical(y, num_classes=3, dtype='float32')

In [9]:
from spacy_transformers import TransformersLanguage, TransformersWordPiecer, TransformersTok2Vec

name = "bert-base-uncased"
nlp = TransformersLanguage(trf_name=name, meta={"lang": "en"})
nlp.add_pipe(nlp.create_pipe("sentencizer"))
nlp.add_pipe(TransformersWordPiecer.from_pretrained(nlp.vocab, name))
nlp.add_pipe(TransformersTok2Vec.from_pretrained(nlp.vocab, name))
print(nlp.pipe_names)  # ['sentencizer', 'trf_wordpiecer', 'trf_tok2vec']

['sentencizer', 'trf_wordpiecer', 'trf_tok2vec']


In [29]:
word_id =[]
for i in tqdm.tqdm(X):
    doc = nlp(i)
    w = doc._.trf_word_pieces
    word_id.append(w)
    
    

100%|██████████| 27480/27480 [52:29<00:00,  8.72it/s]  


In [30]:
model = spacy.load('en_trf_bertbaseuncased_lg')

In [40]:
max_lenght = len(max(word_id, key=len))

In [115]:
test = sequence.pad_sequences(clean_X, maxlen = max_lenght, padding='pre' )

In [43]:
pd.DataFrame(word_id_2).to_csv("word_id.csv", index=None)

In [20]:
def clean_my_text(text):
    lemmatized = []
    text = text.lower()
    tokens = nlp(text)
    for word in tokens:
        if not word.is_stop and word.is_alpha:            
            lemmatized.append(word.lemma_)
    return lemmatized

In [29]:
def preproccess():
    clean_X = []
    for text in tqdm.tqdm(X):
        lemmatized = clean_my_text(text)
        if len(lemmatized) != 0:
            join_string = ' '.join(lemmatized)
            doc = nlp(join_string)
            word_id = doc._.trf_word_pieces
        else:
            word_id = lemmatized        
        clean_X.append(word_id)
    max_lenght = len(max(clean_X, key=len))    
    word_vec_X = sequence.pad_sequences(clean_X, maxlen = max_lenght, padding='pre')
    pd.DataFrame(word_vec_X).to_csv("word_vec_spacy_trans.csv", index=None)
    
    
        

In [30]:
preproccess()

100%|██████████| 27480/27480 [2:30:59<00:00,  3.03it/s]  


In [10]:
word_vec_X = pd.read_csv('word_vec_spacy_trans.csv')
word_vec_X

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,101,5838,2183,102
1,0,0,0,0,0,0,0,0,0,0,...,0,0,101,17111,2080,6517,3335,2624,5277,102
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,101,5795,18917,102
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,101,4357,2681,102
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,101,4124,7085,4149,102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27475,0,0,0,0,0,0,0,0,0,0,...,101,4299,2272,1057,7573,3129,2439,3105,8984,102
27476,0,0,0,0,0,0,0,0,0,0,...,2486,16475,2015,4553,2047,11374,29003,10507,7159,102
27477,0,0,0,0,0,0,0,0,0,0,...,2342,2002,13306,5353,2729,15876,2078,22038,20348,102
27478,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,101,4276,102


In [122]:
vocab_list = ['']
for text in clean_X:
    for word in text:        
        vocab_list.append(word)

# takes care of the duplicates
vocab_list = list(set(vocab_list))

In [17]:
list = [[1 ,2 ,3], [2], [2,2,2,2,2,2,2]]
len(max(list, key=len))

7

In [65]:
word_to_num = {}
num_to_word = {}
for i, word in enumerate(vocab_list):
    num_to_word[i] = word
    word_to_num[vocab_list[i]] = i

In [66]:
word_vec_x = [[word_to_num[word] for word in text] for text in clean_X]

In [67]:
word_vec_X = sequence.pad_sequences(word_vec_x, maxlen = 22, padding='pre' )

In [24]:
clean_x_series = pd.Series(clean_X)

In [25]:
clean_x_series = clean_x_series.apply(pd.Series)

In [26]:
clean_x_series.dropna(axis=0)

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
1234,xo,hey,chick,u,alryt,u,dads,tmoro,sud,sumin,...,week,neva,dun,oot,week,lol,missed,ha,bye,hun
16149,av,ad,realy,gd,day,wiv,ciara,connolly,park,gate,...,new,brighton,fukin,funny,bt,sunburnt,luks,like,drivers,arm


In [71]:
pd.DataFrame(word_vec_X).to_csv("word_vec_X_trans.csv", index=None)

In [23]:
word_vec_X = pd.read_csv('word_vec_X_trans.csv')


In [11]:
word_vec_X = word_vec_X.to_numpy()

In [12]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(word_vec_X, y)

In [86]:
vocab_size = len(vocab_list) + 1 
vocab_size

22735

In [31]:
model = Sequential()
model.add(Embedding(vocab_size, 64, input_length=22))
model.add(Dropout(0.5))
model.add(BatchNormalization())
# model.add(LSTM(1024))
# model.add(Dropout(0.5))
model.add(LSTM(512, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(1, activation='sigmoid'))

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.35))
model.add(BatchNormalization())

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.35))
model.add(BatchNormalization())

model.add(Dense(3, activation='softmax'))

In [33]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics= ['accuracy'])

In [35]:
model.fit(Xtrain, ytrain, epochs=3, batch_size=32, validation_split=0.2)

Train on 16488 samples, validate on 4122 samples
Epoch 1/3
16488/16488 [==============================] - 28s 2ms/sample - loss: 0.6935 - accuracy: 0.6562 - val_loss: 0.5241 - val_accuracy: 0.7196
Epoch 2/3
16488/16488 [==============================] - 23s 1ms/sample - loss: 0.4854 - accuracy: 0.7705 - val_loss: 0.4452 - val_accuracy: 0.7935
Epoch 3/3
16488/16488 [==============================] - 23s 1ms/sample - loss: 0.3969 - accuracy: 0.8261 - val_loss: 0.4614 - val_accuracy: 0.8017


In [36]:
from sklearn.metrics import classification_report

y_pred = model.predict(Xtrain, batch_size=64, verbose=0)
y_pred_bool = np.argmax(y_pred, axis=1)
ytrain_bool = np.argmax(ytrain, axis=1)
print(classification_report(ytrain_bool, y_pred_bool))

              precision    recall  f1-score   support

           0       0.79      0.81      0.80      8362
           1       0.86      0.83      0.85      6477
           2       0.81      0.81      0.81      5771

    accuracy                           0.82     20610
   macro avg       0.82      0.82      0.82     20610
weighted avg       0.82      0.82      0.82     20610



In [37]:
y_pred = model.predict(Xtest, batch_size=64, verbose=0)
y_pred_bool = np.argmax(y_pred, axis=1)
ytest_bool = np.argmax(ytest, axis=1)
print(classification_report(ytest_bool, y_pred_bool))

              precision    recall  f1-score   support

           0       0.65      0.70      0.68      2755
           1       0.78      0.73      0.76      2105
           2       0.70      0.67      0.68      2010

    accuracy                           0.70      6870
   macro avg       0.71      0.70      0.71      6870
weighted avg       0.70      0.70      0.70      6870



### BERT

In [71]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(word_id_2, y)

In [13]:
import bert
from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer

In [14]:
model_dir = "./uncased_L-12_H-768_A-12"

bert_params = bert.params_from_pretrained_ckpt(model_dir)
l_bert = bert.BertModelLayer.from_params(bert_params, name="bert")

In [ ]:
max_seq_len = 41
l_input_ids      = tensorflow.keras.layers.Input(shape=(max_seq_len,), dtype='int32')
# l_token_type_ids = tensorflow.keras.layers.Input(shape=(max_seq_len,), dtype='int32')

# using the default token_type/segment id 0
output = l_bert(l_input_ids) # output: [batch_size, max_seq_len, hidden_size]

cls_out = tensorflow.keras.layers.Lambda(lambda seq: seq[:, 0, :])(output)
cls_out = Dropout(0.5)(cls_out)
# cls_out = BatchNormalization()(cls_out)
logits = Dense(768, activation="tanh")(cls_out)
logits = Dropout(0.5)(logits)
# logits = Dense(128, activation="relu")(logits)
# logits = Dropout(0.5)(logits)
logits = Dense(units=3,activation="softmax")(logits)


model = tensorflow.keras.Model(inputs=l_input_ids, outputs=logits)
model.build(input_shape=(None, max_seq_len))


# model = tensorflowkeras.Model(inputs=l_input_ids, outputs=output)
# model.build(input_shape=(None, max_seq_len))

# provide a custom token_type/segment id as a layer input
# output = l_bert([l_input_ids, l_token_type_ids])          # [batch_size, max_seq_len, hidden_size]
# model = tensorflow.keras.Model(inputs=[l_input_ids, l_token_type_ids], outputs=output)
# model.build(input_shape=[(None, max_seq_len), (None, max_seq_len)])

In [ ]:
bert_ckpt_file   = os.path.join(model_dir, "bert_model.ckpt")
bert.load_stock_weights(l_bert, bert_ckpt_file)

In [ ]:
model.compile(optimizer=tensorflow.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics= ['accuracy'])

In [ ]:
model.fit(Xtrain, ytrain, epochs=5, batch_size=32, validation_split=0.2)